In [2]:
!pip install gpt4all langchain==0.0.342

  Obtaining dependency information for gpt4all from https://files.pythonhosted.org/packages/b3/5b/1f4334f54546605efb878129ecde1a6bbab1710c233440fef2888b56b5c8/gpt4all-2.0.2-py3-none-macosx_10_15_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 5.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


#### download model here https://gpt4all.io/index.html

In [1]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate

In [4]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [2]:
local_path = (
    "./models/gpt4all-falcon-q4_0.gguf"  # replace with your desired local file path
)

In [3]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

In [5]:
# create LLM chain
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [8]:
#basic general LLM knowledge
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

output = llm_chain.run(question)

 
1. Justin Bieber was born on March 1, 1994. 
2. The year he was born was 1994. 
3. In 1994, the NFL team that won the Super Bowl was the San Francisco 49ers.

In [10]:
from langchain.document_loaders import TextLoader

loader = TextLoader("data/subset.txt")
documents = loader.load()

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [12]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter


from langchain.document_loaders import WikipediaLoader
 
# Load content from Wikipedia using WikipediaLoader
loader = WikipediaLoader("Machine_learning")
docs= loader.load()

#split in chunks of 2000 characters - max input size for GPT 2000 and a bit
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = text_splitter.split_documents(docs)

#get embeddings
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')



persist_directory = 'db/chroma_3/'

# Create the vector store
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
template = """Answer the question in your own words from the 
context given to you.
If questions are asked where there is no relevant context available, please answer from 
what you know.

Context: {context}

Human: {question}
Assistant:"""

prompt = PromptTemplate(
input_variables=["context",  "question"], template=template)

In [14]:
#retrieval 
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    verbose=False,
    chain_type_kwargs={
        "verbose": False,
        "prompt": prompt
    }
)

In [15]:
res = qa("What is overfitting")

 Overfitting occurs when the model becomes too complex and fits the training data too closely, resulting in poor generalization performance on new data. This can happen if the model is trained with too much noise or if the model is not able to generalize well enough to new data.